# Aggregating and Sorting 

In this section, we will learn how to summarize records using SQL's `GROUP BY` and `ORDER BY` operators. Along the way we will learn aggregating functions like `SUM`, `COUNT`, `MIN`, `MAX`, and `AVG`. 

# Setup

Download the SQLite database and initalize a SQLite3 connection.

In [1]:
import sqlite3
import pandas as pd
import urllib.request

# download SQLite database and connect to it 
urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

We will continue working with the `WEATHER_MONITOR` table and summarize records using aggregate functions. 

# Aggregate Functions and GROUP BY 

Let's take a look at three fields in the `WEATHER_MONITOR` table. 

In [2]:
sql = """

SELECT REPORT_CODE, REPORT_DATE, RAIN

FROM WEATHER_MONITOR 

"""

pd.read_sql(sql, conn)

,REPORT_CODE,REPORT_DATE,RAIN
0,UVYMMWW,2021-03-20,3.74
1,7VVYE2L,2021-04-10,0.00
2,PJVNOSP,2021-02-26,1.58
3,3B19P7S,2021-05-30,0.00
4,EHVUPGY,2021-04-09,0.00
...,...,...,...
2995,RWG4DY4,2020-11-26,0.00
2996,9A9EQNQ,2020-11-29,2.24
2997,C7ALVIK,2021-01-14,0.00
2998,V660AWC,2021-01-10,2.73


Let's say we wanted to find the total `RAIN` across the entire table. If we remove the `REPORT_CODE` and `REPORT_DATE` fields, and put the `SUM()` around `RAIN`, observe what happens. 

In [3]:
sql = """

SELECT SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

"""

pd.read_sql(sql, conn)

,TOTAL_RAIN
0,1720.78


So we have 1720.78 inches of rain total across the whole table. Let's break up that `TOTAL_RAIN` by `LOCATION_ID`. We can achieve this by selecting the `LOCATION_ID` and performing a `GROUP BY` on it. 

In [4]:
sql = """

SELECT LOCATION_ID, SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID

"""

pd.read_sql(sql, conn)

,LOCATION_ID,TOTAL_RAIN
0,0,43.71
1,1,67.53
2,2,46.76
3,3,38.37
4,4,60.32
5,5,48.63
6,6,26.71
7,7,72.50
8,8,33.21
9,9,30.74


Note how we have sums broken out by `LOCATION_ID` now, or in other words we rolled up that `TOTAL_RAIN` by `LOCATION_ID`. If we wanted to get the total by `LOCATION_ID` and `YEAR`, we can break it up by those two fields/expressions.

In [5]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

,LOCATION_ID,YEAR,TOTAL_RAIN
0,0,2020,24.72
1,0,2021,18.99
2,1,2020,45.02
3,1,2021,22.51
4,2,2020,16.04
...,...,...,...
95,47,2021,14.08
96,48,2020,32.29
97,48,2021,30.09
98,49,2020,13.07


Note also we can use `GROUP BY` with ordinal index for each selected column/expression rather than the column name. Note this uses 1-based indexing. 

In [6]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY 1, 2

"""

pd.read_sql(sql, conn)

,LOCATION_ID,YEAR,TOTAL_RAIN
0,0,2020,24.72
1,0,2021,18.99
2,1,2020,45.02
3,1,2021,22.51
4,2,2020,16.04
...,...,...,...
95,47,2021,14.08
96,48,2020,32.29
97,48,2021,30.09
98,49,2020,13.07


There are other aggregation functions besides `SUM()`. `MIN()` will find the minimum value for a given column while `MAX()` will find the maximum. `AVG()` will calculate the average column while `COUNT()` will count the number of non-null values for that column. Here are all five of these aggregate functions to create a  report summarizing descriptive rain statistics by `LOCATION_ID` and `YEAR`. 

In [7]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 

SUM(RAIN) AS TOTAL_RAIN, 
MIN(RAIN) AS MIN_RAIN,
MAX(RAIN) AS MAX_RAIN,
AVG(RAIN) AS AVG_RAIN, 
COUNT(RAIN) AS COUNT_RAIN

FROM WEATHER_MONITOR 

GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

,LOCATION_ID,YEAR,TOTAL_RAIN,MIN_RAIN,MAX_RAIN,AVG_RAIN,COUNT_RAIN
0,0,2020,24.72,0.00,3.74,0.727059,34
1,0,2021,18.99,0.00,3.74,0.730385,26
2,1,2020,45.02,0.00,4.14,1.324118,34
3,1,2021,22.51,0.00,4.14,0.703438,32
4,2,2020,16.04,0.00,3.68,0.616923,26
...,...,...,...,...,...,...,...
95,47,2021,14.08,0.00,3.00,0.440000,32
96,48,2020,32.29,0.00,3.41,0.787561,41
97,48,2021,30.09,0.00,3.41,0.557222,54
98,49,2020,13.07,-0.12,2.00,0.450690,29


We can also use a `WHERE` filter to only allow certain records to qualify in our aggregations. Below we calculate the total `RAIN` by `YEAR` and `LOCATION_ID`, but only where a `TORNADO` was present.

In [8]:
sql = """

SELECT 
LOCATION_ID, 
strftime('%Y', REPORT_DATE) AS YEAR, 
SUM(RAIN) AS TOTAL_TORNADO_RAIN

FROM WEATHER_MONITOR 

WHERE TORNADO = 1
GROUP BY LOCATION_ID, YEAR

"""

pd.read_sql(sql, conn)

,LOCATION_ID,YEAR,TOTAL_TORNADO_RAIN
0,0,2021,0.61
1,2,2021,NaN
2,3,2021,5.77
3,4,2021,1.40
4,5,2021,2.50
5,7,2021,0.00
6,8,2021,0.72
7,10,2021,4.19
8,12,2021,2.24
9,13,2021,1.47


## Counting Records

If you want to count the number of records in a table, pass the whole record to the `COUNT()` function rather than a specific field. This can be achieved with using an asterisk `*`. 

In [9]:
sql = """

SELECT COUNT(*) AS RECORD_COUNT

FROM WEATHER_MONITOR 
"""

pd.read_sql(sql, conn)

,RECORD_COUNT
0,3000


All the other operations we used previously to slice and filter records can also be used with the `COUNT(*)`. Below we break up the record count by `YEAR`, but only count records where `RAIN` was at least 2 inches. 

In [10]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

COUNT(*) AS RECORD_COUNT

FROM WEATHER_MONITOR 

WHERE RAIN >= 2

GROUP BY YEAR 
"""

pd.read_sql(sql, conn)

,YEAR,RECORD_COUNT
0,2020,208
1,2021,223


## Sorting 

Let's take a look at the query below showing the `TOTAL_RAIN` by `YEAR` and `MONTH`. 

In [11]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH
"""

pd.read_sql(sql, conn)

,YEAR,MONTH,TOTAL_RAIN
0,2020,11,392.22
1,2020,12,433.16
2,2021,1,316.27
3,2021,2,138.07
4,2021,3,129.03
5,2021,4,153.79
6,2021,5,158.24


Notice that the records coincidentally are ordered by `YEAR` ascending and `MONTH` ascending. You should never expect records to come back in any order without an `ORDER BY`, even if the SQL engine has an implementation that gives this impression. This can happen especially if the data is physially stored in a sorted by (e.g. chronologically). 

To enforce an ascending order by `YEAR` and `MONTH`, add an `ORDER BY` operator. 

In [12]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH

ORDER BY YEAR, MONTH
"""

pd.read_sql(sql, conn)

,YEAR,MONTH,TOTAL_RAIN
0,2020,11,392.22
1,2020,12,433.16
2,2021,1,316.27
3,2021,2,138.07
4,2021,3,129.03
5,2021,4,153.79
6,2021,5,158.24


You can also reference the selected expressions using ordinal index. 

In [13]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY 1, 2

ORDER BY 1, 2
"""

pd.read_sql(sql, conn)

,YEAR,MONTH,TOTAL_RAIN
0,2020,11,392.22
1,2020,12,433.16
2,2021,1,316.27
3,2021,2,138.07
4,2021,3,129.03
5,2021,4,153.79
6,2021,5,158.24


If we wanted to have the most recent years displayed first, add the `DESC` keyword to make a given field sort in descending order. 

In [14]:
sql = """

SELECT 
CAST(strftime('%Y', REPORT_DATE) AS INTEGER) AS YEAR, 
CAST(strftime('%m', REPORT_DATE) AS INTEGER) AS MONTH, 

SUM(RAIN) AS TOTAL_RAIN

FROM WEATHER_MONITOR 

GROUP BY YEAR, MONTH

ORDER BY YEAR DESC, MONTH
"""

pd.read_sql(sql, conn)

,YEAR,MONTH,TOTAL_RAIN
0,2021,1,316.27
1,2021,2,138.07
2,2021,3,129.03
3,2021,4,153.79
4,2021,5,158.24
5,2020,11,392.22
6,2020,12,433.16


## EXERCISE 

Complete the query below to find the total, minimum, and maximum snowfall by year. Order on the year descending so the latest year is on the top. 

In [ ]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

? AS TOTAL_SNOW, 
? AS MIN_SNOW,
? AS MAX_SNOW

FROM WEATHER_MONITOR 

? BY ?
? BY ? DESC
"""

pd.read_sql(sql, conn)



### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """

SELECT 
strftime('%Y', REPORT_DATE) AS YEAR, 

SUM(SNOW) AS TOTAL_SNOW, 
MIN(SNOW) AS MIN_SNOW,
MAX(SNOW) AS MAX_SNOW

FROM WEATHER_MONITOR 

GROUP BY YEAR
ORDER BY YEAR DESC
"""

pd.read_sql(sql, conn)